In [ ]:
# NOTES
# Versioning persists after deletion: e.g. I delete version 2, next version will be 3 regardless.
# Send request with json.dumps(data) to wrap json format into a string, otherwise it does not work
# Pausing takes some time and a journey version cannot be resumed unless it is in status "Paused"

In [216]:
import pandas as pd
import requests
import re
import json
import time
from time import sleep
import datetime
from math import *
import sys
import functools
import warnings
warnings.filterwarnings("ignore")

# Logging

In [217]:
import logging

logging.basicConfig(filename='ikea_journey_weights_update.log', level=logging.DEBUG,
                    format="%(asctime)s:%(levelname)s:%(message)s")

# Handle Errors

In [265]:
class RequestError(Exception):
    
    def __init__(self, response, message):
        self.response = response
        self.message = message
        super().__init__(message)

class StatusError(Exception):
    
    def __init__(self, status, message):
        self.status = status
        self.message = message
        super().__init__(message)

# Journey Builder API Credentials

In [219]:
# Retrieve login parameters from config file
with open("config.json") as credentials:
    credentials = json.load(credentials)

client_id = credentials["client_id"]
client_secret = credentials["client_secret"]

# Journey Builder Class

In [266]:
class JourneyBuilder:
    """This class is meant to represent specific journeys within Salesforce Journey Builder API."""
    
    def __init__(self, journey_name):

        self.journey_name = journey_name
        
        # Authentication
        self.client_id = client_id
        self.client_secret = client_secret
        self.auth_url = "https://mc42bdlx7mz5h4np2xxvhsb4scvq.auth.marketingcloudapis.com/v2/token"
        self.auth_headers = {"content-type": "application/json"}
        self.auth_payload = {
            "grant_type": "client_credentials",
            "client_id": self.client_id,
            "client_secret": self.client_secret
        }
        
        self.rest_url = "https://mc42bdlx7mz5h4np2xxvhsb4scvq.rest.marketingcloudapis.com"

        try:
            self.access_token, self.access_token_expiration, self.rest_headers = self.get_access_token()
            if self.access_token is None:
                raise Exception("Request for access token failed.")
        except Exception as e:
            print(e)

    def get_access_token(self):
        """Get access token; associated with the decorator below."""
        try:
            authentication_response = requests.post(
                url=self.auth_url, 
                data=json.dumps(self.auth_payload), 
                headers=self.auth_headers, 
                verify=False)
            authentication_response.raise_for_status()
        except Exception as e:
            print(e)
            return None
        else:
            access_token = authentication_response.json()["access_token"]
            access_token_expiration = authentication_response.json()["expires_in"]
            rest_headers = {"authorization": f"Bearer {access_token}"}
            return access_token, access_token_expiration, rest_headers
    
    class Decorators():
        @staticmethod
        def refresh_token(decorated):
            """Methods with this decorator will refresh the access token if it is expired before running."""
            def wrapper(api,*args,**kwargs):
                if time.time() > api.access_token_expiration:
                    api.access_token, api.access_token_expiration, api.rest_headers = api.get_access_token()
                return decorated(api,*args,**kwargs)
            return wrapper
    
    @Decorators.refresh_token
    def get_metadata(self, *args):
        """Get journey metadata such as id, key, name, version, status etc.."""
        
        # Get all journeys for user
        response = requests.get(
            url=f"{self.rest_url}/interaction/v1/interactions/",
            headers=self.rest_headers,
            verify=False
        )
        # Raise exception if status not in 200s
        pattern = re.compile("20[0-9]")
        match = re.findall(pattern, str(response.status_code))
        if not match:
            raise RequestError(
                response, "get_metadata: Response not in 200s.")
        
        # Return metadata for journey = journey_name
        return [[journey[arg] for journey in json.loads(response.content)["items"]
                 if journey["name"] == self.journey_name][0] for arg in args]
    
    @Decorators.refresh_token
    def change_status(self, action):
        """Change journey status with an action: Pause, Resume, Stop."""
        
        journey_id, journey_version = self.get_metadata("id", "version")
        
        # The API needs to know where it has to perform the action: journey id + version.
        response = requests.post(
            url=f"{self.rest_url}/interaction/v1/interactions/{action}/{journey_id}?versionNumber={journey_version}",
            headers=self.rest_headers,
            verify=False
        )
        # Raise exception if status not in 200s
        pattern = re.compile("20[0-9]")
        match = re.findall(pattern, str(response.status_code))
        if not match:
            raise RequestError(
                response, f"change_status: Response not in 200s.")
    
    @Decorators.refresh_token
    def get_activities(self):
        """Get all journey activities with the journey id."""
        
        journey_id = self.get_metadata("id")[0]

        # Request gets all activities within journey based on id
        response = requests.get(
            url=f"{self.rest_url}/interaction/v1/interactions/{journey_id}",
            headers=self.rest_headers,
            verify=False
        )
        # Raise exception if status not in 200s
        pattern = re.compile("20[0-9]")
        match = re.findall(pattern, str(response.status_code))
        if not match:
            raise RequestError(
                response, "get_activities: Response not in 200s.")   
        return json.loads(response.content)
    
    @Decorators.refresh_token
    def update_version(self, journey_version_updated):
        """Update journey version with updated activities. Status cannot be 'Published'."""
        response = requests.put(
            url=f"{self.rest_url}/interaction/v1/interactions",
            headers=self.rest_headers,
            data=json.dumps(journey_version_updated),
            verify=False
        )
        # Raise exception if status not in 200s
        pattern = re.compile("20[0-9]")
        match = re.findall(pattern, str(response.status_code))
        if not match:
            raise RequestError(
                response, f"update_journey_version: Response not in 200s.")

# Update Journey Weights Func

In [257]:
def update_journey_weights(journey_activities, journey_new_weights):
    
    # journey_new_weights = {"journey_name": "", "email_name": "", "new_weight": 00}
    
    # Iterate through the journey paths / emails
    for email_name in journey_new_weights["email_name"]:
        
        email_name_new_weight = journey_new_weights.loc[journey_new_weights["email_name"] == email_name]["new_weight"].values[0]
        # For each path, iterate through the journey activities
        # Find the activity whose name == path_name / email_name and get the activity key (e.g. "EMAILV2-1")
        # Iterate through the journey activities
        # Find outcome whose next value == activity key (e.g. Corresponding activity == "RANDOMSPLITV2-1")
        # Update that outcome ["arguments"]["percentage"] with "00" + ["metaData"]["label"] with "00%"
        for activity in journey_activities["activities"]:
             if activity["name"] == email_name:
                    key = activity["key"]
        for activity in journey_activities["activities"]:
            for outcome in activity["outcomes"]:
                if "next" in outcome:
                    if outcome["next"] == key:
                        outcome["arguments"]["percentage"] = str(email_name_new_weight)
                        outcome["metaData"]["label"] = str(email_name_new_weight) + "%"
    return journey_activities

# Reading Data

In [269]:
journey_new_weights = pd.read_csv("test_journey_builder.csv", sep=";")
journey_new_weights

,journey_name,email_name,new_weight
0,test_kwr_multilanguage_setup,A,80
1,test,B,10
2,test,C,10
3,test_2910,BIrthday,89
4,test_2910,poule,11


In [270]:
# Filter for testing purposes
journey_new_weights = journey_new_weights.loc[journey_new_weights["journey_name"] == "test_2910"]
journey_new_weights

,journey_name,email_name,new_weight
3,test_2910,BIrthday,89
4,test_2910,poule,11


In [271]:
journey_to_be_updated = JourneyBuilder("test_2910")  # "AbandonedCart Program") # "test_kwr_multilanguage_setup")

In [274]:
journey_id, journey_version, journey_status = journey_to_be_updated.get_metadata("id", "version", "status")
journey_id, journey_version, journey_status

('e1906618-b1e6-4649-a159-1e382cc483c6', 4, 'Published')

In [226]:
journey_to_be_updated.get_activities()

[{'id': 'c953cd3c-72f7-405a-8ee7-62a51d98326f',
  'key': 'RANDOMSPLITV2-1',
  'name': '',
  'description': '',
  'type': 'RANDOMSPLIT',
  'outcomes': [{'key': 'branchResult-1',
    'next': 'EMAILV2-1',
    'arguments': {'percentage': '20'},
    'metaData': {'label': '20%', 'pathName': 'Path 1', 'invalid': False}},
   {'key': 'branchResult-2',
    'next': 'EMAILV2-2',
    'arguments': {'percentage': '80'},
    'metaData': {'label': '80%', 'pathName': 'Path 2', 'invalid': False}}],
  'arguments': {},
  'configurationArguments': {},
  'metaData': {'icon': '/extensions/activities/random-split/images/randomsplit.svg',
   'iconSmall': '/extensions/activities/random-split/images/randomsplit.svg',
   'category': 'flow',
   'isConfigured': True,
   'original_icon': '/extensions/activities/random-split/images/randomsplit.svg',
   'original_iconSmall': '/extensions/activities/random-split/images/randomsplit.svg',
   'original_statsContactIcon': '',
   'statsContactIcon': ''},
  'schema': {'argume

In [275]:
for journey_name in journey_new_weights["journey_name"].unique():
            
    # Get data only for that journey
    journey_new_weights = journey_new_weights.loc[journey_new_weights["journey_name"] == journey_name]
    
    # Instantiate journey object
    journey_to_be_updated = JourneyBuilder(journey_name)
    
    # Get journey version metadata
    journey_id, journey_version, journey_status = journey_to_be_updated.get_metadata("id", "version", "status")
    
    # Conditions depending on status
    if journey_status == "Published":
        
        # The journey cannot be updated with status = "Published"
        # The journey can be updated with status in ("Draft", Paused", "Pausing")
        journey_to_be_updated.change_status("Pause")
        logging.debug(f"Journey {journey_id}, Version {journey_version}, Status {journey_status}, PAUSED SUCCESSFULLY")
        
        # The journey activities should be updated based on the new weihts with status != "Published"
        # Therefore, we get the journey activities after pausing the journey so the status is correct
        journey_version_old = journey_to_be_updated.get_activities()
        journey_version_updated = update_journey_weights(journey_version_old, journey_new_weights)
        journey_to_be_updated.update_version(journey_version_updated)
        logging.debug(f"Journey {journey_id}, Version {journey_version}, Status {journey_status}, UPDATED SUCCESSFULLY")
        
        # The journey needs to be with status = "Paused" to be resumed
        # It takes some time for the journey to pause completely
        sleep(300)
        journey_status = journey_to_be_updated.get_metadata("status")[0]
        if journey_status != "Paused":
            raise StatusError(
                journey_status, f"status = {journey_status}, should be Paused.") 
        
        # Once status = "Paused", then we can resume, otherwise we raise an error
        journey_to_be_updated.change_status("Resume")
        logging.debug(f"Journey {journey_id}, Version {journey_version}, Status {journey_status}, RESUMED SUCCESSFULLY")

    elif journey_status in ["Draft", "Paused"]:
        
        # The journey activities should be updated based on the new weihts with status != "Published"
        journey_version_old = journey_to_be_updated.get_activities()
        journey_version_updated = update_journey_weights(journey_version_old, journey_new_weights)
        journey_to_be_updated.update_version(journey_version_updated)  # I think will not work because of format
        logging.debug(f"Journey {journey_id}, Version {journey_version}, Status {journey_status}, UPDATED SUCCESSFULLY")
    
    else:
        
        logging.debug(f"Journey {journey_id}, Version {journey_version}, Status {journey_status}, IGNORED.")

  journey_name email_name  new_weight
3    test_2910   BIrthday          89
4    test_2910      poule          11
journey_name:  test_2910
journey_new_weights:    journey_name email_name  new_weight
3    test_2910   BIrthday          89
4    test_2910      poule          11
id version status:  e1906618-b1e6-4649-a159-1e382cc483c6 4 Published
status == Published
{'id': '6e818bc7-4070-4991-a68f-9566bd716284', 'key': 'RANDOMSPLITV2-1', 'name': '', 'description': '', 'type': 'RANDOMSPLIT', 'outcomes': [{'key': 'branchResult-1', 'next': 'EMAILV2-1', 'arguments': {'percentage': '5'}, 'metaData': {'label': '5%', 'pathName': 'Path 1', 'invalid': False}}, {'key': 'branchResult-2', 'next': 'EMAILV2-2', 'arguments': {'percentage': '95'}, 'metaData': {'label': '95%', 'pathName': 'Path 2', 'invalid': False}}], 'arguments': {}, 'configurationArguments': {}, 'metaData': {'icon': '/extensions/activities/random-split/images/randomsplit.svg', 'iconSmall': '/extensions/activities/random-split/images/rand

StatusError: status = ['Paused'], should be Paused.

In [ ]:
# since it takes approx 5' to update a live journey, maybe we can update in parallel
# check logging course also

In [ ]:
# INPUT
# 'lang = EN': Path1 = 90%
# 'CA': Path1 = 10%
# 'ES': Path1 = 10%
# None: Path1 = 90%
# 'Remainder': Path1 = 10%

In [51]:
LANGUAGE_MAPPING = {outcome["metaData"]["label"]: outcome["next"] for outcome in json_data["activities"][0]["outcomes"]}
LANGUAGE_MAPPING

{'lang = EN': 'RANDOMSPLITV2-1',
 'CA': 'RANDOMSPLITV2-2',
 'ES': 'RANDOMSPLITV2-3',
 None: 'RANDOMSPLITV2-4',
 'Remainder': 'RANDOMSPLITV2-5'}

In [61]:
for activity in json_data["activities"]:
    if activity["key"] in LANGUAGE_MAPPING.values():
        print(activity["key"], "\n",
              activity["outcomes"][0]["key"], "\n",
              activity["outcomes"][0]["metaData"]["pathName"], "\n", # needs to be name instead of pathName for V2-1
              activity["outcomes"][0]["metaData"]["label"], "\n",
              activity["outcomes"][0]["arguments"]["percentage"], "\n",
              activity["outcomes"][1]["key"], "\n",
              activity["outcomes"][1]["metaData"]["pathName"], "\n",
              activity["outcomes"][1]["metaData"]["label"], "\n",
              activity["outcomes"][1]["arguments"]["percentage"],"\n\n")

RANDOMSPLITV2-4 
 branchResult-1 
 Path 1 
 50% 
 50 
 branchResult-2 
 Path 2 
 50% 
 50 


RANDOMSPLITV2-3 
 branchResult-1 
 Path 1 
 50% 
 50 
 branchResult-2 
 Path 2 
 50% 
 50 


RANDOMSPLITV2-2 
 branchResult-1 
 Path 1 
 50% 
 50 
 branchResult-2 
 Path 2 
 50% 
 50 


RANDOMSPLITV2-5 
 branchResult-1 
 Path 1 
 50% 
 50 
 branchResult-2 
 Path 2 
 50% 
 50 




KeyError: 'pathName'

In [ ]:
"7a3ab8c6-e800-4faa-a867-0b59a4d37809"

In [ ]:
# logic to update weights
# lang label associated with next action name (RANDOMSPLITV2-1)
# Find next activity where key == next best action name of previous code
# Update relevant branch / pathName

In [69]:
# create_version method in case I need it at some point.
    
    @Decorators.refreshToken
    def create_version(self, journey_id, journey_version, new_perc):
        # Update journey version
        journey_activities = self.get_activities_by_id(journey_id)
        new_version_journey_activities = self.update_journey_perc(journey_activities, new_perc, version="new")
        # Create new jounrey version
        r = requests.post(
            url=f'{self.rest_url}/interaction/v1/interactions',
            headers=self.rest_headers,
            data=json.dumps(new_version_journey_activities),
            verify=False
        )
        logging.info(f"Journey: {journey_id, journey_version+1} CREATED SUCCESSFULLY!")
        # Stop old version
        self.change_status(journey_id, journey_version, "stop")
        logging.info(f"Journey: {journey_id, journey_version} STOPPED SUCCESSFULLY!")